In [5]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#imports
import pandas as pd
import numpy as np

In [0]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 300)

In [0]:
#Load in our play by play data
project_dir = "/content/drive/My Drive/nfl"
plays = pd.read_csv(f"{project_dir}/data/pbp_1999_2019.csv")

In [0]:
#Filter out plays that are end of games, replays, challenges, & timeouts
plays = plays.loc[(plays['epa'].notnull()) & (plays['ep']!=0)]
plays.drop(plays[(plays['replay_or_challenge'] == 0) & (plays['desc'].str.contains('Timeout'))].index, inplace=True)

In [0]:
#Get average number of plays in a game
plays_games = plays.groupby(by=['alt_game_id'])['epa'].count()
avg_plays = plays_games.mean()

In [0]:
#Function to generate the exitement of a game - formula from https://sports.sites.yale.edu/game-excitement-index-part-ii, originally by Brian Burke 
def get_gei(game_id):
  game = plays.loc[(plays['alt_game_id']==game_id)]
  #length of game
  num_plays = len(game)
  #normalize excitement index by length of game (longer games aren't necessarily more exciting)
  norm_factor = avg_plays / num_plays
  #Get win probability differences for each play
  excitement_diff = game['home_wp'].diff().abs()
  #Normalization
  gei = norm_factor * excitement_diff.sum()
  return gei


In [0]:
%%time
#Get top 10 most exciting chiefs games

#list of all kc games since 2009
kc_games = plays['alt_game_id'].loc[(plays['alt_game_id'].str.contains('KC')) & (plays['season']>=2009)].unique()
#Make empty df to store game excitement indexes
excitement_df = pd.DataFrame(columns=['game_id','excitement'])
#loop through all kc games, getting excitement index for each
for kc_game in kc_games:
  gei = get_gei(kc_game)
  row = [kc_game, gei]
  game_df = pd.DataFrame([row],columns=['game_id','excitement'])
  excitement_df = excitement_df.append(game_df)

In [0]:
excitement_df.sort_values(by='excitement',ascending=False).head(6)